In [ ]:
import torch
import tensorflow as tf

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import matplotlib.pyplot as plt
import numpy as np
cudnn.benchmark = True
plt.ion()   # interactive mode
# torch.manual_seed(0)

In [ ]:
!pip install torchmetrics

In [ ]:
# Setup the dataset
!wget -nc  https://zenodo.org/record/5108846/files/masonry-images-sensors-2021.zip?download=1
import zipfile
import os
if os.path.exists('data/')==False:
  local_zip = 'masonry-images-sensors-2021.zip?download=1'
  zip_ref   = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('data/')
  zip_ref.close()

In [ ]:
from torchvision.transforms.transforms import Resize

data_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        Resize([100,100])
        ])

data_dir = 'data/masonry-images-sensors-2021'
batch_size = 16

train_set = datasets.ImageFolder(os.path.join(data_dir, 'train'),data_transforms)
test_set = datasets.ImageFolder(os.path.join(data_dir, 'test'),data_transforms)
real_set = datasets.ImageFolder(os.path.join(data_dir, 'real'), data_transforms)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=True, num_workers=2)
real_loader = torch.utils.data.DataLoader(real_set, batch_size=batch_size,shuffle=True, num_workers=2)

classes_labels = ['uncracked', 'cracked']
print('{} samples found for training set'.format(len(train_set)))
print('{} samples found for testing set'.format(len(test_set)))
print('{} samples found for real set'.format(len(real_set)))

total_tr = len(train_set)
total_rl = len(real_set)
total_ts = len(test_set)
tr_batches = len(train_loader)
rl_batches = len(real_loader)
ts_batches = len(test_loader)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

642 samples found for training set
214 samples found for testing set
90 samples found for real set


In [ ]:
# Download the pretrained model
!pip install git+https://github.com/wielandbrendel/bag-of-local-features-models.git

In [ ]:
import bagnets.pytorchnet
pytorch_model = bagnets.pytorchnet.bagnet33(pretrained=True)

In [ ]:
import bagnets.pytorchnet
pytorch_model = bagnets.pytorchnet.bagnet17(pretrained=True)
for param in pytorch_model.parameters():
    param.requires_grad = True
for param in pytorch_model.layer3.parameters():
  param.requires_grad = False
for param in pytorch_model.layer2.parameters():
  param.requires_grad = False
for param in pytorch_model.layer1.parameters():
  param.requires_grad = False

In [ ]:
a = 1024
b= 512
class cls(nn.Module):
  def __init__(self):
    super(cls, self).__init__()
    self.relu = nn.ReLU()
    self.drp = nn.Dropout(0.2)
    self.fc = nn.Linear(2048, a)        
    self.fc2 = nn.Linear(a, b)
    self.fc3 = nn.Linear(b,2)
  def forward(self, x):
    x = x.view(-1, 2048*11*11)
    x = self.relu(x)
    x = self.fc(x)   
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    return x

pytorch_model.fc = cls()
pytorch_model = pytorch_model.to(device)

In [ ]:
pytorch_model = pytorch_model.to(device)

In [ ]:
from torchvision import models
from torchsummary import summary

summary(pytorch_model,(3,100,100))

In [ ]:
model = pytorch_model
from torchmetrics import Accuracy, Precision, Recall, F1Score
loss_fn = nn.CrossEntropyLoss()

train_acc = Accuracy()
real_acc = Accuracy()

optimizer = torch.optim.Adam(model.parameters(),lr=0.004)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

pbar = tf.keras.utils.Progbar(target=tr_batches)

def train_one_epoch():
  running_loss = 0.0
  accuracy = 0.0  
  total_recl = []
  total_prec = []
  total_f1 = []
  for i, data in enumerate(train_loader):
    
      
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      
      outputs = model(inputs)
      _, preds = torch.max(outputs.data, 1)
      
      loss = loss_fn(outputs, labels)
      
      
      loss.backward()
      optimizer.step()         
      
      acc = accuracy_score(preds.cpu(), labels.cpu())
     
      recl = recall_score(preds.cpu(),labels.cpu(),zero_division=0)

      prec = precision_score(preds.cpu(),labels.cpu(),zero_division=0)
      
      f1 = f1_score(preds.cpu(),labels.cpu(),zero_division=0)
      
      accuracy += (preds == labels).sum().item()
      
      running_loss += loss.item() * labels.size(0)
      
      
      total_recl.append(recl.item())
      total_prec.append(prec.item())
      total_f1.append(f1.item())
      pbar.update(i+1, values=[("loss",loss.item()),("acc",acc),("Recall",recl.item()),("Precision",prec.item()),("F1",f1.item())])
  
  total_loss = running_loss / (total_tr)
  total_accuracy = (100 * accuracy / total_tr)
  total_recl = np.mean(total_recl)
  total_f1 =  np.mean(total_f1)
  total_prec =  np.mean(total_prec)
  return total_loss, total_accuracy, total_recl, total_prec, total_f1

In [ ]:
def validation_step(model,val_loader,size):
  pbar2 = tf.keras.utils.Progbar(target=len(val_loader))
  model.train(False)
  model.eval()
  running_vloss = 0.0
  vacc = 0.0
  total_vrecl = []
  total_vprec = []
  total_vf1 = []
  for i, vdata in enumerate(val_loader):
        
        vinputs, vlabels = vdata
        vinputs = vinputs.to(device)
        vlabels = vlabels.to(device)

        voutputs = model(vinputs)
        _, vpreds = torch.max(voutputs.data, 1)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss * vlabels.size(0)
        vacc += (vpreds == vlabels).sum().item()
        real_acc = accuracy_score(vpreds.cpu(), vlabels.cpu())

        vrecl = recall_score(vpreds.cpu(),vlabels.cpu(), zero_division=0)
        vprec = precision_score(vpreds.cpu(),vlabels.cpu(), zero_division=0)
        vf1 = f1_score(vpreds.cpu(),vlabels.cpu(), zero_division=0)

        pbar2.update(i+1, values=[("loss",vloss.item()),("acc",real_acc),("Recall",vrecl),("Precision",vprec),("F1",vf1)])

        total_vrecl.append(vrecl)
        total_vprec.append(vprec)
        total_vf1.append(vf1)

  total_vrecl = np.mean(total_vrecl)
  total_vf1 =  np.mean(total_vf1)
  total_vprec =  np.mean(total_vprec)
  avg_vloss = running_vloss / (size)
  avg_vacc = (100 * vacc / size)

  return avg_vloss, avg_vacc, total_vprec, total_vrecl, total_vf1

In [ ]:
best_t=0
best_r=0
def train_model(model,n_epochs=5):
  
  best_vacc = -100.
  best_acc = -100.
  epoch_number = 0
  
  for epoch in range(n_epochs):

    print(f'Epoch {epoch+1}/{n_epochs}')
    
    model.train(True)
    loss, acc, recl, prec, f1 = train_one_epoch()


In [ ]:
train_model(model,50)

In [ ]:
val_loss = 0.0
val_steps = 0
total = 0
correct = 0
for i, data in enumerate(real_loader, 0):
  with torch.no_grad():
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    loss = loss_fn(outputs, labels)
    val_loss += loss.cpu().numpy()
    val_steps += 1